### Exp: 2 Cluster customers first... Then map each cluster to vehicle

Assumption: Customer comes in cluster big small medium with some noise(somebody crossing da road) clustering them might mean they comes from the same vehicle (or come together from somewhere)

Cross : Create a HDBSCAN cluster then tune it accordingly

Cross OUTPUT : 
- df_cross['cluster'] : the cluster that the that customer belongs to with 
- cluster_cross : the cluster info of cross containin 
    - xy mid mean, min , max - timestamp (start stop avg) and COUNT of cluster size


camera used : mbk-14-12-vehicle 

### Load

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

pd.set_option("display.precision", 10)
CURRENT_CAM = "mbk-14-11"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)
# print(df_vehicle.to_string())
# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())

### Preprocess

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Clustering the customer

In [4]:
df_cross, cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross)
print(Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


           timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
1    2024-04-28 02:00:18  2024-04-28 02:35:01      4           0  1330.6250000000  651.8750000000
5    2024-04-28 02:56:10  2024-04-28 02:57:19      2           4  1802.0000000000  430.5000000000
89   2024-04-28 03:02:30  2024-04-28 03:02:37      2          88  1653.5000000000  483.0000000000
90   2024-04-28 03:02:48  2024-04-28 03:03:17      8          89  1645.1875000000  501.9375000000
60   2024-04-28 03:03:37  2024-04-28 03:03:39      2          59  1750.0000000000  522.0000000000
0    2024-04-28 03:04:07  2024-04-28 16:48:21     38          -1  1661.7105263158  449.1052631579
25   2024-04-28 03:04:57  2024-04-28 03:05:37      2          24  1609.2500000000  484.2500000000
17   2024-04-28 03:07:01  2024-04-28 03:07:04      2          16  2100.7500000000  213.2500000000
18   2024-04-28 03:10:34  2024-04-28 03:11:39      3          17  1538.3333333333  567.8333333333
19   2024-04-28 03:1

/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:963: RuntimeWarning: invalid value encountered in multiply
  dist_mat = dist_mat * strength
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/in

In [5]:
df_reverse, cluster_reverse = Cluster.perform_reverse_clustering(
    df_reverse=df_reverse, params=conf.HDBSCAN_PARAMS_REVERSE
)
print(Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string())

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


           timestamp_min        timestamp_max  count  cluster_id        xmid_mean       ymid_mean
0    2024-04-28 02:00:15  2024-04-28 16:44:53     32          -1  1270.7187500000  569.6093750000
5    2024-04-28 02:05:56  2024-04-28 02:06:56      2           4  1975.5000000000  185.0000000000
6    2024-04-28 02:10:21  2024-04-28 02:11:19      2           5  1875.7500000000  307.7500000000
1    2024-04-28 02:19:42  2024-04-28 02:21:08      2           0  2098.2500000000  143.5000000000
2    2024-04-28 03:02:21  2024-04-28 03:11:45      7           1  1691.2142857143  370.2142857143
3    2024-04-28 03:17:36  2024-04-28 03:19:08      2           2  2099.2500000000  138.2500000000
10   2024-04-28 04:00:13  2024-04-28 04:00:20      2           9  1319.5000000000  527.7500000000
18   2024-04-28 04:06:28  2024-04-28 04:06:57      3          17  1757.6666666667  364.1666666667
19   2024-04-28 04:12:58  2024-04-28 04:19:33      5          18  1417.5000000000  483.2000000000
20   2024-04-28 04:3

/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-co

result: The clustering now seems good enough with time_biased_distance4 metric (care more on time than coordinate and time diff is more than x amount it probably is not from the same group)

TRY assigning vehicle to cluster first ( then เก็บตก large cluster only?)

In [6]:
# tony = Cluster.assign_cross_cluster_to_vehicle_in_lifetime(df_vehicle,cluster_cross,distance_metric="euclidean")
# print(tony[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())

In [7]:
tony = Cluster.assign_cross_cluster_to_vehicle_in_lifetime(
    df_vehicle, cluster_cross, distance_metric="euclidean"
)
# print(tony[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())
kenny = Cluster.assign_reverse_cluster_to_vehicle_in_lifetime(
    tony, cluster_reverse, distance_metric="euclidean"
)
print(
    kenny[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]][
        (kenny["timestamp_unix"] >= 1714284000) & (kenny["timestamp_unix_end"] <= 1714288020)
    ].to_string()
)

                 camera       timestamp_precise        lifetime    xmid   ymid plate_number vehicle_type  count_cross  count_reverse                   cluster_cross_list cluster_reverse_list
1867  mbk-14-11-vehicle 2024-04-28 06:02:29.843   18.8750000000  1388.5  120.5        41360        sedan            2              0                                [133]                   []
1860  mbk-14-11-vehicle 2024-04-28 06:05:35.235   28.7510001659  1406.5  126.0      6กม9678        sedan            5              0                                 [92]                   []
1861  mbk-14-11-vehicle 2024-04-28 06:06:08.106   15.5160000324  1403.0  140.5       มฎ5222          suv            0              0                                   []                   []
1862  mbk-14-11-vehicle 2024-04-28 06:06:28.620   32.1110000610  1535.0  187.0      3กฐ7375    hatchback            2              0                                 [93]                   []
1863  mbk-14-11-vehicle 2024-04-28 06:08:58.4


bad case : 06:47:56.476   516.1789999008  1747.0  200.0       340223       pickup      1714286876          1714287392      9        [87, 111, 151, 152]

TODO: find a better metric function to calculate distances